In [3]:
%env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

import torch
from detectron2.config import get_cfg
from detectron2.engine.defaults import DefaultPredictor
from detectron2.data.datasets import register_coco_instances
from detectron2.data import get_detection_dataset_dicts
from detectron2.data.detection_utils import read_image
from detectron2.data import MetadataCatalog
from detectron2.data import DatasetCatalog
from detectron2.structures import BoxMode

import numpy as np
import os
import json
import cv2
from pycocotools import mask

import json
from detectron2.data.detection_utils import read_image
from PIL import Image
import numpy

import cv2
from matplotlib import pyplot as plt
from detectron2.utils.visualizer import Visualizer, VisImage

from detectron2.config import configurable
from detectron2.modeling.roi_heads.box_head import build_box_head
from detectron2.modeling.roi_heads.roi_heads import StandardROIHeads

from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers, fast_rcnn_inference
from detectron2.structures import Boxes, ImageList, Instances, pairwise_iou
from detectron2.layers import ShapeSpec, batched_nms
from typing import Dict, List, Optional, Tuple
from detectron2.modeling.poolers import ROIPooler
from torch.nn import functional as F

from detectron2.layers import ShapeSpec, batched_nms, cat, cross_entropy, nonzero_tuple
from detectron2.utils.events import get_event_storage
from detectron2.evaluation.coco_evaluation import COCOEvaluator

from detectron2.engine.defaults import DefaultTrainer
import logging
from detectron2.modeling import build_model
import detectron2.data.transforms as T
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.layers import ShapeSpec
import torch

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

class NonCocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        return COCOEvaluator(dataset_name)

env: PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128


In [4]:
training_categ = ['chess', 'skiing', 'weightlifting', 'climbing', 'cricket', 'flying', 'hockey', 'soccer', 'volleyball', 'tennis', 'skateboarding', 'swimming', 'rowing', 'roller skating', 'horse racing', 'steeplechase', 'jogging', 'gymnastics', 'golf', 'diving', 'car racing', 'boxing', 'bowling', 'billiard', 'beach volleyball', 'basketball', 'baseball', 'jumping', 'running', 'acrobatics', 'airplane', 'glider', 'helicopter', 'hot-air_balloon', 'bicycle', 'camper', 'convertible', 'jeep', 'limousine', 'sedan', 'taxi', 'wagon', 'carriage', 'motorcycle', 'bus', 'minibus', 'tram', 'trolleybus', 'road sign', 'traffic police', 'zebra crossing', 'boat', 'ferry', 'gondola', 'motorboat', 'sailing vessel', 'ship', 'yacht', 'sleigh', 'rocket', 'spaceship', 'train', 'car transporter', 'dumper', 'garbage truck', 'lorry', 'pickup', 'tow truck', 'truck', 'van', 'bulldozer', 'digger', 'forklift', 'tractor', 'artist', 'sculptor', 'accordionist', 'piper', 'cellist', 'clarinetist', 'conductor', 'flute player', 'guitar player', 'opera singer', 'percussionist', 'piano player', 'rapper', 'saxophonist', 'singer', 'trombonist', 'trumpeter', 'violinist', 'ballet dancer', 'cameraman', 'clown', 'dancer', 'makeup artist', 'photographer', 'writer', 'figure skating', 'off road motorcycling', 'motorcycle racing', 'baby carriage', 'fire engine', 'fireman', 'police car', 'police helicopter', 'mounted police', 'policeman', 'wheelchair', 'fishing', 'hunting', 'tank', 'hang gliding', 'rhythmic gymnastics', 'horse sleigh', 'ambulance', 'dog sleigh', 'military helicopter', 'police boat', 'motorcycle police', 'soldier', 'double-decker', 'bicycle racing', 'handball', 'armoured personnel carrier', 'military truck', 'rickshaw', 'scooter', 'pole vaulting']

cfg = get_cfg()
cfg.merge_from_file('/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml')
cfg.MODEL.WEIGHTS = 'detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x/137260431/model_final_a54504.pkl'

cfg.DATALOADER.NUM_WORKERS = 1
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 0
cfg.SOLVER.MAX_ITER = 1500 #adjust up if val mAP is still rising, adjust down if overfit
cfg.SOLVER.STEPS = (1000, 1400)
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 16
cfg.TEST.EVAL_PERIOD = 2000
cfg.MODEL.DEVICE='cuda:1'

for folder_name in training_categ:
    try:
        DatasetCatalog.remove(folder_name)
        MetadataCatalog.remove(folder_name)
        print('Update dataset')
    except:
        print('New dataset')
    #register_coco_instances(folder_name,{},'/host/comparison/'+folder_name+'/'+folder_name+'_train.json','/host/comparison/'+folder_name+'/data/')
    register_coco_instances(folder_name,{},'/host/app1/server/uploads/'+folder_name+'_gt.json','/host/app1/server/uploads/'+folder_name+'/')
    
    try:
        del trainer
    except:
        print("Starting...")
    
    ind = 0
    img = DatasetCatalog.get(folder_name)[ind]['file_name']
    #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #v = Visualizer(img,MetadataCatalog.get(folder_name))
    #v.draw_dataset_dict(DatasetCatalog.get(folder_name)[ind])
    print(folder_name)

    #plt.figure(figsize=(15, 15))
    #plt.imshow(v.get_output().get_image())
    #plt.show()
    
    cfg.DATASETS.TRAIN = (folder_name,)
    cfg.DATASETS.TEST = ()
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(MetadataCatalog.get(folder_name).thing_classes)
    
    torch.cuda.init()
        
    trainer = NonCocoTrainer(cfg)
    trainer.resume_or_load(resume=False)
    trainer.train()
    trainer.checkpointer.save(folder_name)

New dataset
Starting...



Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



chess
[02/27 21:30:35 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
    

WARNING [02/27 21:30:35 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[02/27 21:30:35 d2.data.datasets.coco]: Loaded 216 images in COCO format from /host/app1/server/uploads/chess_gt.json
[02/27 21:30:35 d2.data.build]: Removed 0 images with no usable annotations. 216 images left.
[02/27 21:30:35 d2.data.build]: Distribution of instances among all 6 categories:
|  category  | #instances   |  category  | #instances   |   category   | #instances   |
|:----------:|:-------------|:----------:|:-------------|:------------:|:-------------|
|    man     | 54           |   clock    | 81           | chess player | 300          |
| chessboard | 290          |  chessman  | 4690         | water bottle | 55           |
|            |              |            |              |              |              |
|   total    | 5470         |            |              |              |              |
[02/27 21:30:35 d2.data.dataset_mapper]:

model_final_a54504.pkl: 178MB [00:19, 8.97MB/s]                              
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (7, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (7,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (24, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (24,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due

...

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) 

[02/28 00:41:47 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:819: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


[02/28 00:42:49 d2.utils.events]:  eta: 1:19:46  iter: 19  total_loss: 2.093  loss_cls: 0.6701  loss_box_reg: 0.884  loss_mask: 0.5384  loss_rpn_cls: 0.02962  loss_rpn_loc: 0.009889  time: 2.9893  data_time: 2.5142  lr: 0.001  max_mem: 0M
ERROR [02/28 00:42:52 d2.engine.train_loop]: Exception during training:
Traceback (most recent call last):
  File "/detectron2/detectron2/engine/train_loop.py", line 149, in train
    self.run_step()
  File "/detectron2/detectron2/engine/defaults.py", line 494, in run_step
    self._trainer.run_step()
  File "/detectron2/detectron2/engine/train_loop.py", line 267, in run_step
    data = next(self._data_loader_iter)
  File "/detectron2/detectron2/data/common.py", line 234, in __iter__
    for d in self.dataset:
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 521, in __next__
    data = self._next_data()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in _next_data
    retur

SizeMismatchError: Caught SizeMismatchError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/fetch.py", line 32, in fetch
    data.append(next(self.dataset_iter))
  File "/detectron2/detectron2/data/common.py", line 201, in __iter__
    yield self.dataset[idx]
  File "/detectron2/detectron2/data/common.py", line 90, in __getitem__
    data = self._map_func(self._dataset[cur_idx])
  File "/detectron2/detectron2/utils/serialize.py", line 26, in __call__
    return self._obj(*args, **kwargs)
  File "/detectron2/detectron2/data/dataset_mapper.py", line 155, in __call__
    utils.check_image_size(dataset_dict, image)
  File "/detectron2/detectron2/data/detection_utils.py", line 204, in check_image_size
    + " Please check the width/height in your annotati